# C++ Cling Examples for STM32L053Nucleo

![Image](img/head.png)

Load [REMCU](https://remotemcu.com) shared libray

In [ ]:
.L libremcu

Add path with header files

In [ ]:
.I remcu_include/

Including necessary header files. The [“remcu.h”](remcu_include/remcu.h) header must be always included before any MCU header files.

In [ ]:
#include "remcu.h"
#include "stm32l0xx_hal.h"
#include "stm32l0xx_ll_bus.h"

Connection to a debug server. The [**REMCU**](https://remotemcu.com/) library is able to work with [OpenOCD](https://github.com/ilg-archived/openocd/releases/tag/v0.10.0-12-20190422) or [GDB server](https://atollic.com/resources/download/).
There is using the [remcu_connect2OpenOCD](https://remotemcu.com/api-v1-0#remcu_getVersion) function for connecting to the OpenOcd server. The functions have the same set of parameters. The first parameter is an IP address of the debug server. The second one is a port of the debug server. For instance, OpenOCD server print the port in log messages. The third parameter is a connection timeout. Upon a successful connection, the function returns the “true” value, otherwise, it returns “false”.

> Detailed description of other REMCU function you see in [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)

Before that, plug stm32 board with debugger to PC

run openocd utility in command line:
```
openocd -f board/stm32l0discovery.cfg
```
Result:
![Image](img/openocd_unix.png)

In [ ]:
const char * debug_server_ip = "localhost";
const uint16_t default_openocd_port = 6666;
const int timeout_sec = 3; // It can not be negative
bool connection = false;

In [ ]:
connection = remcu_connect2OpenOCD(debug_server_ip, default_openocd_port, timeout_sec)

If you use GDB server (either OpenOCD or ST-Link GDB server or Jlink GDB server  etc.), you uncomment code below and run it:
<details>
  <summary> Jlink windows. Where the port is written <b> (click here)</b></summary>
 
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink.png" >
<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/jlink2.png" >
</details>  
<details>
  <summary> ST link GDB windows. Where the port is written<b> (click here)</b></summary>

<img src="https://raw.githubusercontent.com/remotemcu/remcu_examples/master/wiki/img/run-stling-gdb.png">
</details>

In [ ]:
//const uint16_t default_openocd_gdb_port = 3333;
//const uint16_t default_jlink_port = 2331;
//const uint16_t default_stlink_port = 61234;
//const uint16_t custom_port = ...;
//connection = remcu_connect2GDB(debug_server_ip, default_openocd_gdb_port /*default_jlink_port*/ /*default_stlink_port*/, timeout_sec)

In [ ]:
if(connection == false){
    fprintf(stderr, "1. Check connection to debug server\n");
    fprintf(stderr, "2. Check debug server running\n");
    fprintf(stderr, "3. Check the debugged MCU is correct\n");
    exit(1);
}

If connection were failed and the function returned false. See [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page) 
    
If connection were success... Jupyter console output:
![Image](img/success_connection_for_unix.png)

Reset the MCU (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)):

In [ ]:
remcu_resetRemoteUnit(__HALT)

At this stage the microcontroller clock setting is already configured, this is done through SystemInit() function which is called from startup file (startup_stm32f4xx.s) before to branch to application main. To see the default setting of SystemInit() function, refer to remcu_include/system_stm32f4xx.c file

In [ ]:
SystemInit();

This sample code shows how to use STM32L0xx GPIO HAL API to toggle PA5 Pin 
  (connected to LED2, on STM32L0xx_Nucleo board) in an infinite loop.
   To proceed, 3 steps are required:
  
 STM32L0xx HAL library initialization:
       - Configure the Flash prefetch, Flash preread and Buffer caches
       - In the default implementation , SysTick timer is configured as main source 
         of time base. It is used to generate interrupts at 1ms basis.
       - Low Level Initialization

In [ ]:
HAL_Init();

### GPIO Example
![Image](img/gpio.png)
>The code below have been taken from **IO_Toggle** ([main.c](STM32CubeL0_example_projects/NUCLEO-L053R8/Examples/GPIO/GPIO_IOToggle/Src/main.c)) example of [STM32L0 HAL and Low Layer drivers](https://www.st.com/resource/en/user_manual/dm00113898-description-of-stm32l0-hal-and-low-layer-drivers-stmicroelectronics.pdf)

Clock enable and configure pins:

In [ ]:
  RCC_ClkInitTypeDef RCC_ClkInitStruct;
  RCC_OscInitTypeDef RCC_OscInitStruct;
  
  /* Enable MSI Oscillator */
  RCC_OscInitStruct.OscillatorType = RCC_OSCILLATORTYPE_MSI;
  RCC_OscInitStruct.MSIState = RCC_MSI_ON;
  RCC_OscInitStruct.MSIClockRange = RCC_MSIRANGE_5;
  RCC_OscInitStruct.MSICalibrationValue=0x00;
  RCC_OscInitStruct.PLL.PLLState = RCC_PLL_NONE;
  HAL_RCC_OscConfig(&RCC_OscInitStruct);
  
  
  /* Select MSI as system clock source and configure the HCLK, PCLK1 and PCLK2 
     clocks dividers */
  RCC_ClkInitStruct.ClockType = (RCC_CLOCKTYPE_SYSCLK | RCC_CLOCKTYPE_HCLK | RCC_CLOCKTYPE_PCLK1 | RCC_CLOCKTYPE_PCLK2);
  RCC_ClkInitStruct.SYSCLKSource = RCC_SYSCLKSOURCE_MSI;
  RCC_ClkInitStruct.AHBCLKDivider = RCC_SYSCLK_DIV1;
  RCC_ClkInitStruct.APB1CLKDivider = RCC_HCLK_DIV1;  
  RCC_ClkInitStruct.APB2CLKDivider = RCC_HCLK_DIV1;  
  HAL_RCC_ClockConfig(&RCC_ClkInitStruct, FLASH_LATENCY_0);

In [ ]:
  /* -1- Enable GPIOA Clock (to be able to program the configuration registers) */
  //__HAL_RCC_GPIOA_CLK_ENABLE(); - don't use __HAL_RCC_GPIOA_CLK_ENABLE define
LL_IOP_GRP1_EnableClock(LL_IOP_GRP1_PERIPH_GPIOA);

Configure PA.5 IO in output push-pull mode to drive external LEDs 

In [ ]:
 GPIO_InitTypeDef  GPIO_InitStruct;  
/* -2- */
  GPIO_InitStruct.Pin = (GPIO_PIN_5);
  GPIO_InitStruct.Mode = GPIO_MODE_OUTPUT_PP;
  GPIO_InitStruct.Pull = GPIO_PULLUP;
  GPIO_InitStruct.Speed = GPIO_SPEED_FREQ_HIGH  ;
  
  HAL_GPIO_Init(GPIOA, &GPIO_InitStruct); 

Flash leds on:

In [ ]:
HAL_GPIO_WritePin(GPIOA, GPIO_PIN_5, GPIO_PIN_SET)

<details>
  <summary><b>Result(click to show)</b></summary>

![Image](img/led_on.png)
</details>

Flash leds off:

In [ ]:
HAL_GPIO_WritePin(GPIOA, GPIO_PIN_5, GPIO_PIN_RESET)

If it didn’t happen, you can use the utility functions to detect errors...

### Utility Functions

The ***remcu_getErrorCount***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function returns the number of errors that occurred after connection:

In [ ]:
remcu_getErrorCount()

If no error occurred, result ***0***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)

The ***remcu_isConnected***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function indicates status of the current debug server connection:

In [ ]:
remcu_isConnected()

Result: connected ***true*** , disconnected ***false***

Some of debug adapters can have hardware problems. The ***remcu_debuggerTest*** (see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function enables to detect the such adapters. The function returns a null pointer if the test is successful:

In [ ]:
remcu_debuggerTest()

If no error occurred, result ***nullptr***, otherwise see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)

>If bugs are not discovered and the chip don't responds to driver function call or the result of work was not what you expected, see [Issue](https://github.com/remotemcu/remcu_examples/issues) and [Troubleshooting Page](https://remotemcu.com/troubleshooting-page)


### PWM example
![Image](img/pwm_wave.png)
This example shows how to configure the TIM peripheral in PWM (Pulse Width Modulation) mode.

>The code below have been based on **TIM_PWM_Output** ([main.c](STM32CubeL0_example_projects/NUCLEO-L053R8/Examples/TIM/TIM_PWMOutput/Src/main.c)) example of [STM32L0 HAL and Low Layer drivers](https://www.st.com/resource/en/user_manual/dm00113898-description-of-stm32l0-hal-and-low-layer-drivers-stmicroelectronics.pdf)

GPIOA Configuration: TIM2 CH1 (PA5) in output, push-pull, alternate function mode

In [ ]:
  GPIO_InitStruct.Mode = GPIO_MODE_AF_PP;
  GPIO_InitStruct.Pull = GPIO_PULLDOWN;
  GPIO_InitStruct.Speed = GPIO_SPEED_FREQ_VERY_HIGH;
  GPIO_InitStruct.Alternate = GPIO_AF5_TIM2;
  
  GPIO_InitStruct.Pin = GPIO_PIN_5;
  HAL_GPIO_Init(GPIOA, &GPIO_InitStruct);

Enable the timer peripheral clock:

In [ ]:
LL_APB1_GRP1_EnableClock(LL_APB1_GRP1_PERIPH_TIM2);

Configuration of the TIM peripheral in PWM (pulse width modulation) mode.

The TIM2CLK frequency is set to SystemCoreClock  (Hz), to get TIM2 counter
clock at 16 MHz the Prescaler is computed as following:
   - Prescaler = (TIM2CLK / TIM2 counter clock) - 1

SystemCoreClock is set to 32 MHz for STM32L0xx Devices.

The TIM2 generates 4 PWM signals at 10 KHz:

    TIM2_Output Frequency = TIM2 counter clock/(ARR + 1)
                          = 16 MHz / 1600
                          = 10 KHz
                        
The TIM2 CCR1 register value is equal to 1200, so the TIM2 Channel 1 generates a 
PWM signal with a frequency equal to 10 KHz and a duty cycle equal to 75%:

    TIM2 Channel1 duty cycle = (TIM2_CCR1/ TIM2_ARR + 1)* 100 = 75%
    
Initialize TIMx peripheral as follow:
+ Prescaler = (SystemCoreClock)/160000
+ Period = 1600  (to have an output frequency equal to 10 KHz) - PERIOD_VALUE 
+ ClockDivision = 0
+ Counter direction = Up

In [ ]:
#define  PERIOD_VALUE       (1600 - 1)  /* Period Value  */
#define  PULSE1_VALUE       1200        /* Capture Compare 1 Value  */

/* Timer handler declaration */
TIM_HandleTypeDef    TimHandle;

/* Timer Output Compare Configuration Structure declaration */
TIM_OC_InitTypeDef sConfig;

/* Counter Prescaler value */
uint32_t uwPrescalerValue = 0;
/* Compute the prescaler value to have TIM2 counter clock equal to 16 MHz */
uwPrescalerValue = (SystemCoreClock / 160000) - 1;

bool errorflag = false;

In [ ]:
TimHandle.Instance = TIM2;

TimHandle.Init.Prescaler     = uwPrescalerValue;
TimHandle.Init.Period        = PERIOD_VALUE;
TimHandle.Init.ClockDivision = 0;
TimHandle.Init.CounterMode   = TIM_COUNTERMODE_UP;
if(HAL_TIM_PWM_Init(&TimHandle) != HAL_OK)
{
/* Initialization Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
}

/*##-2- Configure the PWM channels #########################################*/ 
/* Common configuration for all channels */
sConfig.OCMode     = TIM_OCMODE_PWM1;
sConfig.OCPolarity = TIM_OCPOLARITY_HIGH;
sConfig.OCFastMode = TIM_OCFAST_DISABLE;

/* Set the pulse value for channel 1 */
sConfig.Pulse = PERIOD_VALUE;  
if(HAL_TIM_PWM_ConfigChannel(&TimHandle, &sConfig, TIM_CHANNEL_1) != HAL_OK)
{
/* Configuration Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
}

/* Start channel 1 */
if(HAL_TIM_PWM_Start(&TimHandle, TIM_CHANNEL_1) != HAL_OK)
{
/* Starting Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
}

In [ ]:
errorflag

Now you can control the duty cycle and adjust the brightness of the board LED.

Change the "**Pulse**" field of the sConfig structure and run functions below:

In [ ]:
sConfig.Pulse = 0;
if(HAL_TIM_PWM_ConfigChannel(&TimHandle, &sConfig, TIM_CHANNEL_1) != HAL_OK)
{
/* Configuration Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
}
/* Start channel 1 */
if(HAL_TIM_PWM_Start(&TimHandle, TIM_CHANNEL_1) != HAL_OK)
{
/* Starting Error */
    fprintf(stderr,"Error\n");
    errorflag = true;
}

Create the slider elements to easily change  the brightness of the board LED.
> First install [**xwidgets**](https://github.com/jupyter-xeus/xwidgets) or make sure you have it
<details>
  <summary>The kernel of notebook should be  <b>C++14</b> (clck here)</summary>
    
![Image](img/Cpp14.png)  

</details>

In [ ]:
#include "xwidgets/xslider.hpp"
xw::slider<int> sliderBrightness_Green;

sliderBrightness_Green.min = 0;
sliderBrightness_Green.max = PERIOD_VALUE;

sliderBrightness_Green.display();

XOBSERVE(sliderBrightness_Green, value, [](const auto& Brightness) {
    sConfig.Pulse = Brightness.value;
    if(HAL_TIM_PWM_ConfigChannel(&TimHandle, &sConfig, TIM_CHANNEL_1) != HAL_OK)
    {
    /* Configuration Error */
        fprintf(stderr,"Error\n");
        errorflag = true;
    }
    /* Start channel 1 */
    if(HAL_TIM_PWM_Start(&TimHandle, TIM_CHANNEL_1) != HAL_OK)
    {
    /* Starting Error */
        fprintf(stderr,"Error\n");
        errorflag = true;
    }
});

### Extra Utility Function

The ***ReMCU_getVersion***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function returns a pointer to a char array containing information about the REMCU lib versions

In [ ]:
remcu_getVersion()

The ***ReMCU_disconnect***(see [“remcu.h”](remcu_include/remcu.h) or [API reference](https://remotemcu.com/api-v1-0)) function disconnects from the debug server. It is finalizing work with REMCU Library.

In [ ]:
remcu_disconnect()